In [60]:
%pip install pyspark
%pip install seaborn
%pip install matplotlib

You should consider upgrading via the '/Users/igli/Spark-Capstone-Project/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/igli/Spark-Capstone-Project/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/igli/Spark-Capstone-Project/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [61]:
from pyspark.sql import SparkSession

In [62]:
spark = SparkSession \
    .builder \
    .config("spark.driver.memory", "10g") \
    .appName("local") \
    .getOrCreate()

In [63]:
members_df = spark.read.csv("data/members_v3.csv", header=True, inferSchema=True)
members_df.show(5)

+--------------------+----+---+------+--------------+----------------------+
|                msno|city| bd|gender|registered_via|registration_init_time|
+--------------------+----+---+------+--------------+----------------------+
|Rb9UwLQTrxzBVwCB6...|   1|  0|  null|            11|              20110911|
|+tJonkh+O1CA796Fm...|   1|  0|  null|             7|              20110914|
|cV358ssn7a0f7jZOw...|   1|  0|  null|            11|              20110915|
|9bzDeJP6sQodK73K5...|   1|  0|  null|            11|              20110915|
|WFLY3s7z4EZsieHCt...|   6| 32|female|             9|              20110915|
+--------------------+----+---+------+--------------+----------------------+
only showing top 5 rows



In [64]:
members_df.printSchema()

root
 |-- msno: string (nullable = true)
 |-- city: integer (nullable = true)
 |-- bd: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- registered_via: integer (nullable = true)
 |-- registration_init_time: integer (nullable = true)



In [65]:
transactions_df = spark.read.csv("data/transactions/churn_comp_refresh/transactions_v2.csv", header=True, inferSchema=True)
transactions_df.show(5)

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|++6eU4LsQ3UQ20ILS...|               32|               90|            298|               298|            0|        20170131|              20170504|        0|
|++lvGPJOinuin/8es...|               41|               30|            149|               149|            1|        20150809|              20190412|        0|
|+/GXNtXWQVfKrEDqY...|               36|               30|            180|               180|            1|        20170303|              20170422|        0|
|+/w1UrZwyka4C9oNH...|               36|            

In [66]:
transactions_df.printSchema()

root
 |-- msno: string (nullable = true)
 |-- payment_method_id: integer (nullable = true)
 |-- payment_plan_days: integer (nullable = true)
 |-- plan_list_price: integer (nullable = true)
 |-- actual_amount_paid: integer (nullable = true)
 |-- is_auto_renew: integer (nullable = true)
 |-- transaction_date: integer (nullable = true)
 |-- membership_expire_date: integer (nullable = true)
 |-- is_cancel: integer (nullable = true)



In [67]:
transactions_df.count()

1431009

In [68]:
user_logs_df = spark.read.csv("data/user_logs/churn_comp_refresh/user_logs_v2.csv", header=True, inferSchema=True)
user_logs_df.show(5)

+--------------------+--------+------+------+------+-------+-------+-------+----------+
|                msno|    date|num_25|num_50|num_75|num_985|num_100|num_unq|total_secs|
+--------------------+--------+------+------+------+-------+-------+-------+----------+
|u9E91QDTvHLq6NXjE...|20170331|     8|     4|     0|      1|     21|     18|  6309.273|
|nTeWW/eOZA/UHKdD5...|20170330|     2|     2|     1|      0|      9|     11|  2390.699|
|2UqkWXwZbIjs03dHL...|20170331|    52|     3|     5|      3|     84|    110| 23203.337|
|ycwLc+m2O0a85jSLA...|20170331|   176|     4|     2|      2|     19|    191|  7100.454|
|EGcbTofOSOkMmQyN1...|20170331|     2|     1|     0|      1|    112|     93| 28401.558|
+--------------------+--------+------+------+------+-------+-------+-------+----------+
only showing top 5 rows



In [69]:
train_df = spark.read.csv("data/train/churn_comp_refresh/train_v2.csv", header=True, inferSchema=True)
train_df.show(5)

+--------------------+--------+
|                msno|is_churn|
+--------------------+--------+
|ugx0CjOMzazClkFzU...|       1|
|f/NmvEzHfhINFEYZT...|       1|
|zLo9f73nGGT1p21lt...|       1|
|8iF/+8HY8lJKFrTc7...|       1|
|K6fja4+jmoZ5xG6By...|       1|
+--------------------+--------+
only showing top 5 rows



In [70]:
logs_agg = user_logs_df.groupBy('msno').agg({'num_25': 'sum', 'num_50': 'sum', 'num_75': 'sum', 'num_985': 'sum', 'num_100': 'sum', 'num_unq': 'sum', 'total_secs': 'sum'})
logs_agg.show(5)

+--------------------+------------+-----------+-----------+------------+-----------+------------+------------------+
|                msno|sum(num_100)|sum(num_25)|sum(num_75)|sum(num_unq)|sum(num_50)|sum(num_985)|   sum(total_secs)|
+--------------------+------------+-----------+-----------+------------+-----------+------------+------------------+
|qufKUjlo3SOny5C/6...|        1489|          4|          7|         574|          2|           8|        367033.421|
|ORUaiH4z66kfVaKGv...|        1851|         66|         31|        1564|         32|          44|446735.52499999997|
|43522HpkDYxZI7k9V...|         593|         72|          9|         550|         11|           4|154447.03999999998|
|nSmTz9Kn4H6Ij9rvI...|        1504|        725|        264|        2186|        325|         425|        545537.587|
|met2zqqAa6vxyrybR...|        1169|        344|         56|        1165|         97|          40|        313319.601|
+--------------------+------------+-----------+-----------+-----

In [71]:
logs_agg = logs_agg.withColumn('sum(num_total)', sum([logs_agg['sum(num_25)'], logs_agg['sum(num_50)'], logs_agg['sum(num_75)'], logs_agg['sum(num_985)'], logs_agg['sum(num_100)']]))
logs_agg.show(5)

+--------------------+------------+-----------+-----------+------------+-----------+------------+------------------+--------------+
|                msno|sum(num_100)|sum(num_25)|sum(num_75)|sum(num_unq)|sum(num_50)|sum(num_985)|   sum(total_secs)|sum(num_total)|
+--------------------+------------+-----------+-----------+------------+-----------+------------+------------------+--------------+
|qufKUjlo3SOny5C/6...|        1489|          4|          7|         574|          2|           8|        367033.421|          1510|
|ORUaiH4z66kfVaKGv...|        1851|         66|         31|        1564|         32|          44|446735.52499999997|          2024|
|43522HpkDYxZI7k9V...|         593|         72|          9|         550|         11|           4|154447.03999999998|           689|
|nSmTz9Kn4H6Ij9rvI...|        1504|        725|        264|        2186|        325|         425|        545537.587|          3243|
|met2zqqAa6vxyrybR...|        1169|        344|         56|        1165|    

In [72]:
data = train_df.join(logs_agg, 'msno')
data.show(5)

+--------------------+--------+------------+-----------+-----------+------------+-----------+------------+------------------+--------------+
|                msno|is_churn|sum(num_100)|sum(num_25)|sum(num_75)|sum(num_unq)|sum(num_50)|sum(num_985)|   sum(total_secs)|sum(num_total)|
+--------------------+--------+------------+-----------+-----------+------------+-----------+------------+------------------+--------------+
|++0/NopttBsaAn6qH...|       0|         104|         21|         17|         115|          8|           7|         28450.268|           157|
|++4RuqBw0Ss6bQU4o...|       0|           9|         14|          0|          24|          1|           0|          2362.332|            24|
|++6P09mCSJSh+Ft2p...|       0|          12|          0|          0|          14|          2|           0|          2994.947|            14|
|++JHjhFuSV7upQUju...|       0|           2|          1|          1|           5|          1|           1|           854.165|             6|
|++UrWpg7IekI

In [73]:
from pyspark.ml.feature import VectorAssembler

cols = [col for col in data.drop('msno').drop('is_churn').columns]

assembler = VectorAssembler(inputCols=cols, outputCol="features")
output = assembler.transform(data)

output = output.withColumn('label', output['is_churn'])

output.show(5)

+--------------------+--------+------------+-----------+-----------+------------+-----------+------------+------------------+--------------+--------------------+-----+
|                msno|is_churn|sum(num_100)|sum(num_25)|sum(num_75)|sum(num_unq)|sum(num_50)|sum(num_985)|   sum(total_secs)|sum(num_total)|            features|label|
+--------------------+--------+------------+-----------+-----------+------------+-----------+------------+------------------+--------------+--------------------+-----+
|++0/NopttBsaAn6qH...|       0|         104|         21|         17|         115|          8|           7|         28450.268|           157|[104.0,21.0,17.0,...|    0|
|++4RuqBw0Ss6bQU4o...|       0|           9|         14|          0|          24|          1|           0|          2362.332|            24|[9.0,14.0,0.0,24....|    0|
|++6P09mCSJSh+Ft2p...|       0|          12|          0|          0|          14|          2|           0|          2994.947|            14|[12.0,0.0,0.0,14....

In [74]:
output.count()

754551

In [75]:
from pyspark.ml.classification import LogisticRegression

trainingData, testingData = output.randomSplit([0.8, 0.2])

lr = LogisticRegression(maxIter=10, regParam=0.02, elasticNetParam=0.8)

lrModel = lr.fit(trainingData)

In [76]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

prediction_df = lrModel.transform(testingData)

ev = MulticlassClassificationEvaluator()
print('accuracy ', ev.evaluate(prediction_df))

accuracy  0.8683867491775505
